In [1]:
%pip install -q -U  langchain langchain-community  langchainhub \
                    google-generativeai langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00


In [3]:
import json

from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

In [5]:
def json_load(dir):
  with open(dir, 'r') as f:
    result = json.load(f)
  return result

In [6]:
# Get API key from here: https://docs.tavily.com/docs/welcome

tools = [TavilySearchResults(max_results=1,
                             api_wrapper=TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY))]

In [17]:
prompt = hub.pull("hwchase17/react")

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [23]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [18]:
# llm = genai.GenerativeModel('gemini-pro')
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85,
                             google_api_key=GEMINI_API_KEY)

In [19]:
agent = create_react_agent(llm, tools, prompt)

In [20]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [21]:
agent_executor.invoke({"input": "what is ReAct as an improvement for prompt writing?"})



> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: what is ReAct as an improvement for prompt writing?[{'url': 'https://www.width.ai/post/react-prompting', 'content': 'Some of the results may surprise you!\nDecember 27, 2023\nIn-Depth Guide to Patient Record Summarization With Large Language Models (With Examples)\nExplore how we create comprehensive patient record summaries using a state-of-the-art pipeline with language-image models and large language models.\nDecember 19, 2023\nHow to Train a Powerful & Local Ai Assistant Chatbot With Data Distillation from GPT-3.5-Turbo\nRun GPT style models on local laptops and edge devices using GPT4All\nDecember 14, 2023\nDo Users Prefer Chatbots or Menu-Based Formats? 1. Provide a ReAct Instruction Prompt for Reasoning and Action Planning\nAn example ReAct prompt with its components is shown below:\nThe prompt must contain four key pieces of information for the LLM:\nA decent chunk of this stems from the id

{'input': 'what is ReAct as an improvement for prompt writing?',
 'output': 'ReAct is an improvement for prompt writing as it allows users to write prompts in a structured and efficient way. It provides a framework for writing prompts that includes a "reason" step, an "action" step, and an "observation" step. This framework helps users to think through the problem they are trying to solve and to write prompts that are more likely to be successful.'}